In [19]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import os, sys, getpass
user = getpass.getuser()
sys.path.append(f'/home/{user}/codes/hpe_library/')
from lib_import import *
from my_utils import *
os.chdir(f'/home/{user}/codes/MotionBERT')
import scipy.io

root_3dhp = f'/home/{user}/Datasets/HAAI/3DHP'
source = 'original' # 
root_original = os.path.join(root_3dhp, source)
train_folder = os.path.join(root_original, 'train')
test_folder = os.path.join(root_original, 'test')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
def convert_intrinsic_from_mm_to_pixel(sensor_size, focal_length, pixel_aspect, center_offset, sensor_pixels_x, sensor_pixels_y):
    # Calculate pixel sizes
    pixel_size_x = sensor_size[0] / sensor_pixels_x
    pixel_size_y = sensor_size[1] * pixel_aspect / sensor_pixels_y

    # Calculate focal length in pixels
    fx = focal_length / pixel_size_x
    fy = focal_length / pixel_size_y

    # Calculate center offset in pixels
    center_offset_pixels_x = center_offset[0] / pixel_size_x
    center_offset_pixels_y = center_offset[1] / pixel_size_y

    cx, cy = sensor_pixels_x/2 + center_offset_pixels_x, sensor_pixels_y/2 + center_offset_pixels_y
    intrinsic = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
    return intrinsic

In [21]:
# Test video info
video_info = {'train': {}, 'test': {}}

for subject in os.listdir(test_folder):
    if 'TS' not in subject:
        continue
    video_info['test'][subject] = {}
    num_frame = len(os.listdir(f'/home/hrai/Datasets/HAAI/3DHP/original/test/{subject}/imageSequence'))
    img_list = natsorted(glob(f'/home/hrai/Datasets/HAAI/3DHP/original/test/{subject}/imageSequence/*.jpg'))
    img = cv2.imread(img_list[0])
    H, W, C = img.shape
    video_info['test'][subject]['num_frame'] = num_frame
    video_info['test'][subject]['W'] = W
    video_info['test'][subject]['H'] = H
print(video_info['test'])

# Test camera parameters
cam_params = {
    'test_cam_1_4': {
        'origin': np.array([3427.28, 1387.86, 309.42]),
        'up': np.array([-0.208215, 0.976233, 0.06014]),
        'right': np.array([0.000575281, 0.0616098, -0.9981]),
        'sensor_size': np.array([10, 10]), # in mm
        'focal_length': 7.32506, # in mm
        'pixel_aspect': 1.00044, # y / x
        'center_offset': np.array([-0.0322884, 0.0929296]) # in mm (positive values move right and down)
    },
    'test_cam_5_6': {
        'origin': np.array([-2104.3074, 1038.6707, -4596.6367]),
        'up': np.array([0.025272345, 0.995038509, 0.096227370]),
        'right': np.array([-0.939647257, -0.009210289, 0.342020929]),
        'sensor_size': np.array([10, 5.625]), # in mm
        'focal_length': 8.770747185, # in mm
        'pixel_aspect': 0.993236423, # 
        'center_offset': np.array([-0.104908645, 0.104899704]) # in mm (positive values move right and down)
    }
}

for key in cam_params.keys():
    up = cam_params[key]['up']
    right = cam_params[key]['right']
    forward = np.cross(up, right)
    R = np.array([right, up, forward]).T #np.array([right, -up, forward])
    t = cam_params[key]['origin']/1000
    C = -R@t
    #extrinsic = np.hstack((R, t.reshape(-1, 1)))
    #print(extrinsic.shape)
    cam_params[key]['R'] = R
    cam_params[key]['t'] = t
    cam_params[key]['C'] = C
    #print(R, t, C)

{'TS2': {'num_frame': 6080, 'W': 2048, 'H': 2048}, 'TS5': {'num_frame': 320, 'W': 1920, 'H': 1080}, 'TS4': {'num_frame': 6009, 'W': 2048, 'H': 2048}, 'TS1': {'num_frame': 6151, 'W': 2048, 'H': 2048}, 'TS3': {'num_frame': 5839, 'W': 2048, 'H': 2048}, 'TS6': {'num_frame': 492, 'W': 1920, 'H': 1080}}


### Testset

In [22]:
cam_dict = {}
for subject in ['TS1', 'TS2', 'TS3', 'TS4', 'TS5', 'TS6']:
    if subject in ['TS1', 'TS2', 'TS3', 'TS4']: key = 'test_cam_1_4' 
    elif subject in ['TS5', 'TS6']: key = 'test_cam_5_6'
    
    R = cam_params[key]['R']
    t = cam_params[key]['t']
    C = cam_params[key]['C']
    W = video_info['test'][subject]['W']
    H = video_info['test'][subject]['H']
    num_frames = video_info['test'][subject]['num_frame']
    
    extrinsic = extrinsic = np.hstack((R, t.reshape(-1, 1)))
    intrinsic = convert_intrinsic_from_mm_to_pixel(cam_params[key]['sensor_size'], 
                                                   cam_params[key]['focal_length'], 
                                                   cam_params[key]['pixel_aspect'], 
                                                   cam_params[key]['center_offset'], 
                                                   W, 
                                                   H)
    
    cam_param = {'R': R, 't': t, 'C': C, 'intrinsic': intrinsic, 'extrinsic': extrinsic, 'num_frames': num_frames, 'W': W, 'H': H} 
    
    cam_dict[subject] = cam_param

savepkl(cam_dict, f'/home/{user}/codes/MotionBERT/custom_codes/Dataset_generation/3dhp/3dhp_test_cam_params.pkl')

### Trainset

In [29]:
# train camera parameters
f = open(os.path.join(train_folder, 'S1', 'Seq1', 'camera.calibration'), 'r')
lines = f.readlines()
name = -1
cam_dict = {}


for line in lines:
    item = line.strip().split(' ')
    item_type = item[0]
    #print(item)
    if item_type == 'name':
        name = f'cam{item[-1]}'
        cam_dict[name] = {}
    elif item_type == 'intrinsic':
        fx = float(item[3])
        cx = float(item[5])
        fy = float(item[8])
        cy = float(item[9])
        intrinsic = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
        cam_dict[name]['intrinsic'] = intrinsic
    elif item_type == 'extrinsic':
        R = np.array([[float(item[3]), float(item[4]), float(item[5])], [float(item[7]), float(item[8]), float(item[9]),], [float(item[11]), float(item[12]), float(item[13])]])
        t = np.array([float(item[6]), float(item[10]), float(item[14])])
        C = -R.T@t
        #print(R.shape, t.shape)
        cam_dict[name]['R'] = R
        cam_dict[name]['t'] = t
        cam_dict[name]['C'] = C
    elif item_type == 'size':
        cam_dict[name]['W'] = float(item[-2])
        cam_dict[name]['H'] = float(item[-1])
        
savepkl(cam_dict, f'/home/{user}/codes/MotionBERT/custom_codes/Dataset_generation/3dhp/3dhp_train_cam_params.pkl')